In [1]:
%run functions.ipynb

np.set_printoptions(precision=3)
np.random.seed(42)

%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
X_train, X_test, y_train, y_test = load_train_and_test_parts()
print("Train and test sizes: {} {}".format(X_train.shape, X_test.shape))
print("(1, 0) labels count in train test: {} {}".format((np.count_nonzero(y_train==1), np.count_nonzero(y_train==0)), 
                                                        (np.count_nonzero(y_test==1), np.count_nonzero(y_test==0))))

Train and test sizes: (726, 12179) (243, 12179)
(1, 0) labels count in train test: (289, 437) (104, 139)


In [3]:
n_iter = 3
cv = 4
n_jobs = 3
scoring = 'roc_auc'

In [4]:
nsc_params = {'shrink_threshold' : np.linspace(0, 5, 50)}
clf_nsc = RandomizedSearchCV(NearestCentroidWrapper(), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, iid=False, param_distributions=nsc_params)  
clf_nsc = fit_clf_print_scores(clf_nsc, X_train, y_train, X_test, y_test)

NearestCentroidWrapper(metric='euclidean', shrink_threshold=3.979591836734694)


,ROC AUC,Precision,Recall,F1
train,0.682,0.601,0.647,0.623
test,0.634,0.583,0.577,0.580


In [5]:
clf_lda = fit_clf_print_scores(LinearDiscriminantAnalysis(shrinkage='auto', solver='lsqr'), X_train, y_train, X_test, y_test)

,ROC AUC,Precision,Recall,F1
train,1.000,1.000,1.000,1.000
test,0.796,0.753,0.615,0.677


In [6]:
logistic_params = {'C' : np.linspace(0.01, 0.6, 50), 'penalty' : ['l1'], 'solver' : ['liblinear']}
tree_params = {'n_estimators' : [1000, 2500, 5000], 'max_depth': [4, 6, 8], 'min_samples_leaf' : [20, 30]}

In [7]:
logit_cv = RandomizedSearchCV(LogisticRegression(), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, iid=False, param_distributions=logistic_params) 
tree_cv = RandomizedSearchCV(RandomForestClassifier(), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, iid=False, param_distributions=tree_params) 

clf_logit = fit_clf_print_scores(logit_cv, X_train, y_train, X_test, y_test)
clf_forest = fit_clf_print_scores(tree_cv, X_train, y_train, X_test, y_test)

/home/stachu/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


LogisticRegression(C=0.35918367346938773, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


,ROC AUC,Precision,Recall,F1
train,0.997,0.989,0.958,0.974
test,0.814,0.745,0.731,0.738


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


,ROC AUC,Precision,Recall,F1
train,0.965,0.944,0.765,0.845
test,0.779,0.743,0.500,0.598


In [8]:
elastic_params = {'l1_ratio' : np.linspace(0.001, 1, 50), 'alpha' : np.linspace(0.0001, 0.5, 100)}
elastic_cv = RandomizedSearchCV(SGDClassifier(loss='log', penalty='elasticnet', max_iter=5000, tol=1e-3), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=elastic_params)
clf = fit_clf_print_scores(elastic_cv, X_train, y_train, X_test, y_test)

SGDClassifier(alpha=0.06574343434343434, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.18448979591836737, learning_rate='optimal', loss='log',
       max_iter=5000, n_iter=None, n_iter_no_change=5, n_jobs=None,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)


,ROC AUC,Precision,Recall,F1
train,0.896,0.984,0.208,0.343
test,0.834,0.960,0.231,0.372


In [9]:
svm_params = {'C' : np.linspace(0.01, 0.6, 50), 'kernel' : ['poly', 'rbf', 'sigmoid']}
svm_cv = RandomizedSearchCV(SVC(probability=True, gamma='scale'), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=svm_params)

clf_svm = fit_clf_print_scores(svm_cv, X_train, y_train, X_test, y_test)

SVC(C=0.2146938775510204, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


,ROC AUC,Precision,Recall,F1
train,1.000,1.000,1.000,1.000
test,0.800,0.719,0.663,0.690


In [10]:
from sklearn.svm import LinearSVC

svm_params = {'C' : np.linspace(0.01, 0.6, 50)}
svm_cv = RandomizedSearchCV(LinearSVC(penalty='l1', dual=False, max_iter=10000), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=svm_params)

clf_svm = fit_clf_print_scores(svm_cv, X_train, y_train, X_test, y_test)

LinearSVC(C=0.2146938775510204, class_weight=None, dual=False,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=10000, multi_class='ovr', penalty='l1', random_state=None,
     tol=0.0001, verbose=0)


/home/stachu/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,ROC AUC,Precision,Recall,F1
train,1.000,1.000,1.000,1.000
test,0.781,0.700,0.673,0.686


In [ ]:
rlr_params = {'C' : np.linspace(0.1, 5, 50), 'n_estimators' : [100, 250, 500, 1000, 2500], 'n_variables' : [100, 250, 500, 1000, 2500]}
rlr_cv = RandomizedSearchCV(RandomLogisticsRegressions(penalty='l1'), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=rlr_params)

clf_rlr = fit_clf_print_scores(rlr_cv, X_train, y_train, X_test, y_test)